# Random Forest

**Decision trees** involve the **greedy selection** of the **best split** point from the dataset at each step.

This algorithm makes decision trees susceptible to **high variance** if they are not pruned. This high variance can be harnessed and reduced by creating **multiple trees with different samples** of the training dataset (different views of the problem) and combining their predictions. This approach is called **bootstrap aggregation or bagging for short**.

A **limitation of bagging** is that the **same greedy algorithm** is used to create each tree, meaning that it is likely that the same or **very similar split** points will be chosen in each tree making the **different trees very similar** (trees will be correlated). This, in turn, **makes their predictions similar, mitigating the variance originally sought.**

We can **force the decision trees to be different by limiting the features (rows)** that the greedy algorithm can evaluate at each split point **when creating the tree**. This is called the **Random Forest algorithm**.

**Like bagging**, **multiple samples** of the training dataset are taken and a **different tree trained** on each. The difference is that at each point a split is made in the data and added to the tree, only a **fixed subset of attributes can be considered.**

For classification problems, the type of problems we will look at in this tutorial, the number of attributes to be considered for the split is limited to the square root of the number of input features.

The result of this one small change are **trees that are more different** from each other (uncorrelated) resulting predictions that are more diverse and a combined prediction that often **has better performance that single tree** or bagging alone.





We are creating a random forest regressor, although the same code can be slightly modified to create a classifier. 

The parameters that define our random forest are:

**x:** independent variables of training set.<br>
**y:** the corresponding dependent variables necessary for supervised learning  <br>
**n_trees :** number of uncorrelated trees we ensemble to create the random forest. <br>
**n_features:** the number of features to sample and pass onto each tree, this is where feature bagging happens. It can either be sqrt, log2 or an integer. In case of sqrt, the number of features sampled to each tree is square root of total features and log base 2 of total features in case of log2.<br>
**sample_size:** the number of rows randomly selected and passed onto each tree. This is usually equal to total number of rows but can be reduced to increase performance and decrease correlation of trees in some cases<br>
**depth:** depth of each decision tree. Higher depth means more number of splits which increases the over fitting tendency of each tree but since we are aggregating several uncorrelated trees, over fitting of individual trees hardly bothers the whole forest.<br>
**min_leaf:** minimum number of rows required in a leaf node. Reduces the chances of over fitting<br>
**seed**: Seed for pseudo random number generator. <br>

We have already discussed Decision Tree Regressor in this [notebook](https://github.com/jyotipmahes/Implementation-of-ML-algos-in-Python/blob/master/Decision%20Tree%20Regression%20.ipynb). Random Forest is a slight extension of this. Please refer to it for detailed explanation.

### 1. Initializing Random Forest Regressor

We will start by creating as class for RandomForest and initialize it. Note we will not create separate fit function for the sake of brevity and do it during initialization. We start by initializing `x`, `y`, `n_trees`, `n_features`, `sample_sz`, `depth` and `min_leaf` and `seed` as discussed above. We also call a function called `self.create_tree()` `n_trees` number of times to create `n_trees`. We will look at the `self.create_tree()` function definition next.

In [1]:
class RandomForest():
    def __init__(self, x, y, n_trees, n_features, sample_sz, depth=10, min_leaf=5, seed=10):
        np.random.seed(seed)
        if n_features == 'sqrt':
            self.n_features = int(np.sqrt(x.shape[1]))
        elif n_features == 'log2':
            self.n_features = int(np.log2(x.shape[1]))
        else:
            self.n_features = n_features
        print(self.n_features, "sha: ",x.shape[1])    
        self.x, self.y, self.sample_sz, self.depth, self.min_leaf  = x, y, sample_sz, depth, min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)]

### 2. Creating trees
`create_tree` function helps us create a set of decision trees. Here are things we do in this function:
1. We shuffle the rows and select rows equal to `sample_sz`.
2. We randomly select feature columns equal to `n_features`.
3. Now we call DecisionTree constructor to create decision tree with selected feature columns, selected rows, given `depth` and `min_leaf`.

Implementation of DecisionTree is discussed in details in previous [notebook](https://github.com/jyotipmahes/Implementation-of-ML-algos-in-Python/blob/master/Decision%20Tree%20Regression%20.ipynb). We will quickly revisit the implementation next.

In [2]:
def create_tree(self):
    idxs = np.random.permutation(len(self.y))[:self.sample_sz]
    f_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
    return DecisionTree(self.x.iloc[idxs], self.y[idxs], self.n_features, f_idxs,
                idxs=np.array(range(self.sample_sz)),depth = self.depth, min_leaf=self.min_leaf)

### 3. Implementation of Decision Tree Regressor
Implementation of Decision Tree Regressor is already discussed in detail in [notebook](https://github.com/jyotipmahes/Implementation-of-ML-algos-in Python/blob/master/Decision%20Tree%20Regression%20.ipynb). Here is the brief steps:
1. Implement function to calculate **mean squared error** using `std_agg` which is the splitting criterion for regression.
2. Initialize a Decision Tree with given inputs.
3. Look for best split across columns/features using `find_varsplit()` and `find_better_split()`. Create new binary split based on the results of finding best split.
4. Defining properties such as `is_leaf`, `split_name` and `split_col` for ease of implementation.
5. Implementing `predict_row` function which transverse the tree add find the mean of the leaf node corresponding to test data features. `predict` is a wrapper to predict for all test data points.

In [3]:
def std_agg(cnt, s1, s2): 
    return (s2/cnt) - (s1/cnt)**2

class DecisionTree():
    def __init__(self, x, y, n_features, f_idxs,idxs,depth=5, min_leaf=5):
        self.x, self.y, self.idxs, self.min_leaf, self.f_idxs = x, y, idxs, min_leaf, f_idxs
        self.depth = depth
        self.n_features = n_features
        self.n, self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for i in self.f_idxs: self.find_better_split(i)
        if self.is_leaf: return
        x = self.split_col
        lhs = np.nonzero(x<=self.split)[0]
        rhs = np.nonzero(x>self.split)[0]
        lf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        rf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        self.lhs = DecisionTree(self.x, self.y, self.n_features, lf_idxs, 
                                self.idxs[lhs], depth=self.depth-1, min_leaf=self.min_leaf)
        self.rhs = DecisionTree(self.x, self.y, self.n_features, rf_idxs, 
                                self.idxs[rhs], depth=self.depth-1, min_leaf=self.min_leaf)

    def find_better_split(self, var_idx):
        x, y = self.x.values[self.idxs,var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)
        sort_y,sort_x = y[sort_idx], x[sort_idx]
        rhs_cnt,rhs_sum,rhs_sum2 = self.n, sort_y.sum(), (sort_y**2).sum()
        lhs_cnt,lhs_sum,lhs_sum2 = 0,0.,0.

        for i in range(0,self.n-self.min_leaf-1):
            xi,yi = sort_x[i],sort_y[i]
            lhs_cnt += 1; rhs_cnt -= 1
            lhs_sum += yi; rhs_sum -= yi
            lhs_sum2 += yi**2; rhs_sum2 -= yi**2
            if i<self.min_leaf or xi==sort_x[i+1]:
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_std*lhs_cnt + rhs_std*rhs_cnt
            if curr_score<self.score: 
                self.var_idx,self.score,self.split = var_idx,curr_score,xi

    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]

    @property
    def is_leaf(self): return self.score == float('inf') or self.depth <= 0 
    
    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
        return t.predict_row(xi)
    

### 4. Implementing predict function for random forest
Since we have the predict function implementation at Decision Tree level, implementing predict function for random forest is very easy. We just **call the predict function on all Decision Trees and take a mean** of their values. We can use other functions such as **median, mode or some custom functions** if we wish show. Here is the implementation.

In [4]:
def predict(self, x):
    return np.mean([t.predict(x) for t in self.trees], axis=0)

## Putting everything together

In [5]:
import math
import numpy as np

class RandomForest():
    def __init__(self, x, y, n_trees, n_features, sample_sz, depth=10, min_leaf=5):
        np.random.seed(12)
        if n_features == 'sqrt':
            self.n_features = int(np.sqrt(x.shape[1]))
        elif n_features == 'log2':
            self.n_features = int(np.log2(x.shape[1]))
        else:
            self.n_features = n_features
        print(self.n_features, "sha: ",x.shape[1])    
        self.x, self.y, self.sample_sz, self.depth, self.min_leaf  = x, y, sample_sz, depth, min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)]

    def create_tree(self):
        idxs = np.random.permutation(len(self.y))[:self.sample_sz]
        f_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        return DecisionTree(self.x.iloc[idxs], self.y[idxs], self.n_features, f_idxs,
                    idxs=np.array(range(self.sample_sz)),depth = self.depth, min_leaf=self.min_leaf)
        
    def predict(self, x):
        return np.mean([t.predict(np.array(x)) for t in self.trees], axis=0)

def std_agg(cnt, s1, s2): 
    return (s2/cnt) - (s1/cnt)**2

class DecisionTree():
    def __init__(self, x, y, n_features, f_idxs,idxs,depth=10, min_leaf=5):
        self.x, self.y, self.idxs, self.min_leaf, self.f_idxs = x, y, idxs, min_leaf, f_idxs
        self.depth = depth
        self.n_features = n_features
        self.n, self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for i in self.f_idxs: self.find_better_split(i)
        if self.is_leaf: return
        x = self.split_col
        lhs = np.nonzero(x<=self.split)[0]
        rhs = np.nonzero(x>self.split)[0]
        lf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        rf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        self.lhs = DecisionTree(self.x, self.y, self.n_features, lf_idxs, self.idxs[lhs], depth=self.depth-1, min_leaf=self.min_leaf)
        self.rhs = DecisionTree(self.x, self.y, self.n_features, rf_idxs, self.idxs[rhs], depth=self.depth-1, min_leaf=self.min_leaf)

    def find_better_split(self, var_idx):
        x, y = self.x.values[self.idxs,var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)
        sort_y,sort_x = y[sort_idx], x[sort_idx]
        rhs_cnt,rhs_sum,rhs_sum2 = self.n, sort_y.sum(), (sort_y**2).sum()
        lhs_cnt,lhs_sum,lhs_sum2 = 0,0.,0.

        for i in range(0,self.n-self.min_leaf-1):
            xi,yi = sort_x[i],sort_y[i]
            lhs_cnt += 1; rhs_cnt -= 1
            lhs_sum += yi; rhs_sum -= yi
            lhs_sum2 += yi**2; rhs_sum2 -= yi**2
            if i<self.min_leaf or xi==sort_x[i+1]:
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_std*lhs_cnt + rhs_std*rhs_cnt
            if curr_score<self.score: 
                self.var_idx,self.score,self.split = var_idx,curr_score,xi

    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]

    @property
    def is_leaf(self): return self.score == float('inf') or self.depth <= 0 
    

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
        return t.predict_row(xi)


**Note this code is based on Fastai [Machine learning course](https://www.fast.ai/2018/09/26/ml-launch/)**

## Checking our implementation

In [6]:
from sklearn.datasets import load_boston
import pandas as pd
bos = load_boston()
x = bos.data
y = bos.target
x_df = pd.DataFrame(x, columns=bos.feature_names)

In [7]:
reg = RandomForest(x_df.iloc[:,:-1], y, n_trees=10, n_features='sqrt', sample_sz=len(y), depth=10, min_leaf=5)

3 sha:  12


In [8]:
pred = reg.predict(x_df.iloc[:,:-1])

In [9]:
sum((pred - y)**2)/len(y)

9.919806291006816

### Comparing to sklearn implementation

In [10]:
from sklearn.ensemble import RandomForestRegressor
skreg = RandomForestRegressor(min_samples_leaf=5, max_depth=10, max_features='sqrt', n_estimators=10)

In [11]:
skreg.fit(x,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [12]:
pred = skreg.predict(x)
sum((pred - y)**2)/len(y)

9.999090363190955

**We can see that our implementation is very similar to sklearn implementation in terms of prediction.**

## Pros and Cons
**Pros**
1. This algorithm can solve both type of problems i.e. classification and regression and does a decent estimation at both fronts.
2. One of benefits of Random forest which excites me most is, the power of handle large data set with higher dimensionality. Further, the model outputs Importance of variable, which can be a very handy feature (on some random data set).
3. It has methods for balancing errors in data sets where classes are imbalanced. The capabilities of the above can be extended to unlabeled data, leading to unsupervised clustering, data views and outlier detection.
4. **Out of Bag Error**:Random Forest involves sampling of the input data with replacement called as bootstrap sampling. Here one third of the data is not used for training and can be used to testing. These are called the out of bag samples. Error estimated on these out of bag samples is known as out of bag error. Study of error estimates by Out of bag, gives evidence to show that the out-of-bag estimate is as accurate as using a test set of the same size as the training set. Therefore, using the out-of-bag error estimate removes the need for a set aside test set.
 

**Cons**
1. It surely does a good job at classification but not as good as for regression problem as it does not give precise continuous nature predictions. In case of regression, it doesn’t predict beyond the range in the training data, and that they may over-fit data sets that are particularly noisy.
2. Random Forest can feel like a black box approach for statistical modelers. You have very little control on what the model does. You can at best try different parameters and random seeds!
 

## Random Forest Classifier
Implementation of Random Forest Classifier is very similar to Regressor we saw above. We just need to change the Decision Tree classifier similar to this [notebook](https://github.com/jyotipmahes/Implementation-of-ML-algos-in-Python/blob/master/Decision%20Tree%20Classifier.ipynb) and slightly modify the Random Forest implementation to accommodate and act as a classifier. Here is the slight changes we make:
1. Change the split criterion as Gini Index
2. Prediction is done by maximum vote.
3. We are not implementing min_leaf condition here as we are using our previous implementation of decision tree classifier. 

Here is the code for **Randomforest classifier**.

In [13]:
import math
import numpy as np
from scipy.stats import mode

class RandomForestClassifier():
    def __init__(self, x, target, n_trees, n_features, sample_sz=None, depth=10):
        if sample_sz == None: sample_sz = x.shape[0]
        np.random.seed(12)
        if n_features == 'sqrt':
            self.n_features = int(np.sqrt(x.shape[1]))
        elif n_features == 'log2':
            self.n_features = int(np.log2(x.shape[1]))
        else:
            self.n_features = n_features
        print(self.n_features, "sha: ",x.shape[1])    
        self.x, self.target, self.sample_sz, self.depth  = x, target, sample_sz, depth
        self.trees = [self.create_tree() for i in range(n_trees)]

    def create_tree(self):
        idxs = np.random.permutation(self.x.shape[0])[:self.sample_sz]
        f_idxs = np.random.permutation(self.x.shape[1]-1)[:self.n_features]
        f_idxs = list(f_idxs)
        f_idxs.append(self.x.shape[1]-1)
        f_idxs = np.array(f_idxs)
        clf = DecisionTree(self.depth)
        clf.fit(self.x.iloc[idxs,f_idxs], self.target)
        return clf
        
    def predict(self, x):
        return mode([t.predict(x) for t in self.trees], axis=0)[0][0]


class DecisionTree:

    def __init__(self, max_depth = 5, depth = 1):
        self.max_depth = max_depth
        self.depth = depth
        self.left = None
        self.right = None
    
    def fit(self, data, target):
        self.data = data
        self.target = target
        self.columns = self.data.columns.tolist()
        self.columns.remove(target)
        if self.depth <= self.max_depth:
            self.__validate_data()
            self.impurity_score = self.__calculate_impurity_score(self.data[self.target])
            self.criteria, self.split_feature, self.information_gain = self.__find_best_split()
            if self.criteria is not None and self.information_gain > 0: self.__create_branches()

    
    def __create_branches(self):
        self.left = DecisionTree(max_depth = self.max_depth, 
                                 depth = self.depth + 1)
        self.right = DecisionTree(max_depth = self.max_depth, 
                                 depth = self.depth + 1)
        left_rows = self.data[self.data[self.split_feature] <= self.criteria] 
        right_rows = self.data[self.data[self.split_feature] > self.criteria] 
        self.left.fit(data = left_rows, target = self.target)
        self.right.fit(data = right_rows, target = self.target)
    
    def __calculate_impurity_score(self, data):
        if data is None or data.empty: return 0
        gini_impurity = 1.
        classes = np.unique(data)
        n_vals = len(data)
        for cl in classes:
            gini_impurity -= np.square(np.sum(data==cl)/n_vals)  
        return gini_impurity
    
    def __find_best_split(self):
        best_split = {}
        for col in self.columns:
            information_gain, split = self.__find_best_split_for_column(col)
            if split is None: continue
            if not best_split or best_split["information_gain"] < information_gain:
                best_split = {"split": split, "col": col, "information_gain": information_gain}

        return best_split.get("split"), best_split.get("col"), best_split.get("information_gain")

    def __find_best_split_for_column(self, col):
        x = self.data[col]
        unique_values = x.unique()
        if len(unique_values) == 1: return None, None
        information_gain = None
        split = None
        for val in unique_values:
            left = x <= val
            right = x > val
            left_data = self.data[left]
            right_data = self.data[right]
            left_impurity = self.__calculate_impurity_score(left_data[self.target])
            right_impurity = self.__calculate_impurity_score(right_data[self.target])
            score = self.__calculate_information_gain(left_count = len(left_data),
                                                      left_impurity = left_impurity,
                                                      right_count = len(right_data),
                                                      right_impurity = right_impurity)
            if information_gain is None or score > information_gain: 
                information_gain = score 
                split = val
        return information_gain, split
    
    def __calculate_information_gain(self, left_count, left_impurity, right_count, right_impurity):
        return self.impurity_score - ((left_count/len(self.data)) * left_impurity + \
                                      (right_count/len(self.data)) * right_impurity)

    def predict(self, data):
        return np.array([self.__flow_data_thru_tree(row) for _, row in data.iterrows()])

    def __validate_data(self):
        non_numeric_columns = self.data[self.columns].select_dtypes(include=
                                ['category', 'object', 'bool']).columns.tolist()
        if(len(set(self.columns).intersection(set(non_numeric_columns))) != 0):
            raise RuntimeError("Not all columns are numeric")

    def __flow_data_thru_tree(self, row):
        if self.is_leaf_node: return self.probability
        tree = self.left if row[self.split_feature] <= self.criteria else self.right
        return tree.__flow_data_thru_tree(row)
        
    @property
    def is_leaf_node(self): return self.left is None

    @property
    def probability(self): 
        return mode(self.data[self.target])[0][0]


### Checking our implementation on iris dataset

In [14]:
from sklearn.datasets import load_iris
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

In [15]:
clf = RandomForestClassifier(iris_df, 'target', 10, 'sqrt')

2 sha:  5


In [16]:
pred = clf.predict(iris_df)
print("Accuracy %.3f" %(np.sum(pred==iris.target)*1.0/len(iris.target)))

Accuracy 1.000


**We can see that our implementation of the classifier is correct.**